In [1]:
import modin.pandas as pd

In [2]:
col_names = pd.read_csv('./DATA/columns.csv', sep='\t')
col_names.head()

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()



,Orig,Svcg
0,Credit Score,Loan Sequence Number
1,First Payment Date,Monthly Reporting Period
2,First Time Homebuyer Flag,Current Actual UPB
3,Maturity Date,Current Loan Delinquency Status
4,Metropolitan Statistical Area (MSA) Or Metropo...,Loan Age


In [3]:
orig_prefix = './DATA/sample_orig_'
svcg_prefix = './DATA/sample_svcg_'
years = [y for y in range(2010,2021)]

In [4]:
deldf_subs = []
for year in years:
    print("Retrieving monthly data for loans of year " + str(year))
    month_df = pd.read_csv(svcg_prefix+str(year)+'.txt',sep='|',names=list(col_names['Svcg']))
    
    tf1 = month_df['Loan Age'] < 12
    tf2 = month_df['Delinquency Due to Disaster'] != 'Y'
    tf3 = ~month_df['Current Loan Delinquency Status'].isin(['R','XX','   '])
    tf = tf1 & tf2 & tf3
    month_df = month_df[tf].copy()
    
    month_df['Current Loan Delinquency Status'] = month_df['Current Loan Delinquency Status'].astype(int)
    agg_dict = {'Current Loan Delinquency Status':sum, 
            'Monthly Reporting Period':'count'}
    deldf_sub = month_df.groupby('Loan Sequence Number').agg(agg_dict)
    col_dct = {'Monthly Reporting Period':'NumMonths'}
    deldf_sub.reset_index(inplace=True)
    deldf_sub.rename(columns=col_dct, inplace=True)
    
    deldf_sub = deldf_sub[deldf_sub['NumMonths']==12].copy()
    if len(deldf_sub) == 0:
        continue
    
    deldf_sub['Loan Deliquency Within Year'] = True

    tf = deldf_sub['Current Loan Delinquency Status'] == 0
    deldf_sub.loc[tf, 'Loan Deliquency Within Year'] = False
    deldf_subs.append(deldf_sub)

Retrieving monthly data for loans of year 2010
Retrieving monthly data for loans of year 2011
Retrieving monthly data for loans of year 2012
Retrieving monthly data for loans of year 2013
Retrieving monthly data for loans of year 2014
Retrieving monthly data for loans of year 2015
Retrieving monthly data for loans of year 2016
Retrieving monthly data for loans of year 2017
Retrieving monthly data for loans of year 2018
Retrieving monthly data for loans of year 2019
Retrieving monthly data for loans of year 2020


To request implementation, send an email to feature_requests@modin.org.


In [5]:
delinq_df = pd.concat(deldf_subs)[['Loan Sequence Number','Loan Deliquency Within Year']]
delinq_df.head()

,Loan Sequence Number,Loan Deliquency Within Year
0,F110Q1000008,False
2,F110Q1000064,False
3,F110Q1000072,False
4,F110Q1000080,False
5,F110Q1000096,False


In [6]:
orig_subs = []
for year in years:
    print("Retrieving origination data for loans of year " + str(year))
    orig_sub = pd.read_csv(orig_prefix+str(year)+'.txt',sep='|',names=list(col_names['Orig']))
    orig_subs.append(orig_sub)

Retrieving origination data for loans of year 2010
Retrieving origination data for loans of year 2011
Retrieving origination data for loans of year 2012
Retrieving origination data for loans of year 2013
Retrieving origination data for loans of year 2014
Retrieving origination data for loans of year 2015
Retrieving origination data for loans of year 2016
Retrieving origination data for loans of year 2017
Retrieving origination data for loans of year 2018
Retrieving origination data for loans of year 2019
Retrieving origination data for loans of year 2020


In [7]:
orig_df = pd.concat(orig_subs)
orig_df.head()

,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,Original Combined Loan-to-Value (CLTV),Original Debt-to-Income (DTI) Ratio,...,Original Loan Term,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-HARP Loan Sequence Number,Program Indicator,HARP Indicator,Property Valuation Method,Interest Only (I/O) Indicator
0,804,201004,N,204003,17860.0,0,1,P,39,20,...,360,2,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,9,NaN
1,786,201004,9,204003,NaN,0,1,P,50,43,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,9,NaN
2,816,201003,N,204002,44220.0,0,1,P,56,27,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,9,NaN
3,783,201003,N,204002,45820.0,0,1,P,80,36,...,360,2,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,9,NaN
4,667,201003,N,204002,17900.0,0,1,P,80,20,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,9,NaN


In [8]:
delinq_df['Loan Sequence Number'].nunique()

396338

In [9]:
orig_df['Loan Sequence Number'].nunique()

525000

In [10]:
training_dat = delinq_df.merge(orig_df, on='Loan Sequence Number', how='left')
training_dat.head()

,Loan Sequence Number,Loan Deliquency Within Year,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,...,Original Loan Term,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-HARP Loan Sequence Number,Program Indicator,HARP Indicator,Property Valuation Method,Interest Only (I/O) Indicator
0,F110Q1000008,False,804,201004,N,204003,17860.0,0,1,P,...,360,2,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,9,NaN
1,F110Q1000064,False,816,201003,N,204002,44220.0,0,1,P,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,9,NaN
2,F110Q1000072,False,783,201003,N,204002,45820.0,0,1,P,...,360,2,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,9,NaN
3,F110Q1000080,False,667,201003,N,204002,17900.0,0,1,P,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,9,NaN
4,F110Q1000096,False,799,201003,N,204002,NaN,0,1,P,...,360,1,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,9,NaN


In [11]:
len(training_dat)

396338

In [12]:
training_dat['Loan Deliquency Within Year'].value_counts()

False    387901
True       8437
Name: Loan Deliquency Within Year, dtype: int64

In [13]:
training_dat['Program Indicator'] = training_dat['Program Indicator'].astype(str)

In [15]:
training_dat.to_pickle('./PROCESSED/training_dat.pkl')

In [ ]:
# Drop month diff > 12